In [1]:
import requests
import pandas
import geopandas
import shapely

# 0. User defined variables

In [2]:
url='http://serveisoberts.gencat.cat/api/equipaments?&exact_matching=&categories.tema=Administraci%C3%B3%20p%C3%BAblica&categories.subtema=Ens%20locals&categories.subtema2=Ajuntaments&adv=true'

# 1. Read data

In [3]:
# not working because:
# DriverError: '/vsimem/b90a75d3a81f4494ac7992a7be08428d' not recognized as a supported file format.

# geopandas.read_file(url)

In [4]:
response=requests.get(url)

In [5]:
data=response.json()

# 2. Manipulate data

## 2.1. Create a smaller dictionary with only relevant fields

In [6]:
relevant_fields=['id','titol','utx','uty','phone']
dictionary={}
for field in relevant_fields:
    dictionary[field]=[]

# iterate over items to generate relevant dictionary
# for item in data['items']:
for i in range(4):
    item=data['items'][i]
    
    dictionary['id'].append(item['id'])
    dictionary['titol'].append(item['titol'])
    dictionary['utx'].append(item['utm']['utmx'])
    dictionary['uty'].append(item['utm']['utmy'])
    dictionary['phone'].append(','.join([element for element in item['telefons'] if len(element) > 3]))
                               
print(dictionary)  

{'id': ['547406', '546929', '547152', '547038'], 'titol': ['Ajuntament de Torredembarra', 'Ajuntament de Guimerà', 'Ajuntament de Llanars', 'Ajuntament de la Morera de Montsant'], 'utx': [365388.698, 348544.428, 445939.895, 316614.249], 'uty': [4556194.781, 4602980.611, 4685578.774, 4568606.642], 'phone': ['977-640025', '973-303038', '972-740361', '977-827112']}


## 2.2. Convert dictionary into pandas dataframe

In [7]:
df=pandas.DataFrame.from_dict(dictionary)
df.head()

,id,titol,utx,uty,phone
0,547406,Ajuntament de Torredembarra,365388.698,4556194.781,977-640025
1,546929,Ajuntament de Guimerà,348544.428,4602980.611,973-303038
2,547152,Ajuntament de Llanars,445939.895,4685578.774,972-740361
3,547038,Ajuntament de la Morera de Montsant,316614.249,4568606.642,977-827112


## 2.3. Convert pandas dataframe to geopandas dataframe

In [8]:
geometry=[shapely.geometry.Point(xy) for xy in zip(df.utx,df.uty)]
crs={'init':'epsg:4326'}
gdf=geopandas.GeoDataFrame(df,crs=crs,geometry=geometry)
gdf.head()

,id,titol,utx,uty,phone,geometry
0,547406,Ajuntament de Torredembarra,365388.698,4556194.781,977-640025,POINT (365388.698 4556194.781)
1,546929,Ajuntament de Guimerà,348544.428,4602980.611,973-303038,POINT (348544.428 4602980.611)
2,547152,Ajuntament de Llanars,445939.895,4685578.774,972-740361,POINT (445939.895 4685578.774)
3,547038,Ajuntament de la Morera de Montsant,316614.249,4568606.642,977-827112,POINT (316614.249 4568606.642)


# 3. Save to shapefile

In [9]:
gdf.to_file("locations.shp")